In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext autotime|

## This is the first phase of manual corrections on instruct_qa


In [13]:
negatives_to_positives = {
    "alpaca-7b": [200, 1258, 1376, 1764, 2934, 3187, 3485, 3547, 3579], 
    "flan-t5-xxl": [3074, 3366, 156, 2789], 
    "gpt-3.5-turbo": [1021, 1147, 3510],
    "llama-2-7b-chat": [701, 1258, 1937, 2096, 2895, 2934, 3485]
}

In [14]:
positives_to_negatives = {
    "alpaca-7b": [441, 1147, 1290, 3045, 3366], 
    "flan-t5-xxl": [640, 1932, 2087, 3407, 3566], 
    "gpt-3.5-turbo": [335, 441, 468, 513, 732, 1258, 1290, 2087, 2292, 2416, 2690, 3045, 3321, 3366, 3433, 3566, 3579],
    "llama-2-7b-chat": [335, 2532, 2988, 3045, 3566, 3579]
}

In [15]:
total = 0 

for index, value in negatives_to_positives.items(): 
    total+=len(value)
    
for index, value in positives_to_negatives.items(): 
    total+=len(value)

In [17]:
import numpy as np 
import pandas as pd 

In [18]:
df = pd.read_csv('instruct_qa_full.csv')
df.head()

,id,model_name,dataset,question,answer,ground_truth,prompt,human_verdict
0,68,alpaca-7b,hotpot_qa,who is the younger brother of The episode gues...,Brian Doyle-Murray is the younger brother of t...,Bill Murray,Please answer the following question given the...,incorrect
1,263,alpaca-7b,hotpot_qa,"Which character does this protagonist, who sec...",Romeo.,Tybalt,Please answer the following question given the...,incorrect
2,279,alpaca-7b,hotpot_qa,What type of magazine is Eugene Habecker the c...,Christianity Today magazine.,evangelical Christian periodical,Please answer the following question given the...,incorrect
3,317,alpaca-7b,hotpot_qa,"What Kentucky county has a population of 60,31...",Oldham County.,Oldham County,Please answer the following question given the...,correct
4,335,alpaca-7b,hotpot_qa,Mamie Gummer played the role of Nancy Crozier ...,The Good Wife (CBS),CBS,Please answer the following question given the...,correct


In [ ]:
df['model_name'].value_counts()

In [20]:
df = df[df['dataset'] == 'natural_questions']

In [ ]:
df['human_verdict'].value_counts()

In [23]:
df['human_verdict'] = df['human_verdict'].apply(lambda x: 'incorrect' if x =='refuse-to-answer' else x) 

In [ ]:
df['human_verdict'].value_counts()

In [ ]:
counter = 0 
for key, ids in negatives_to_positives.items(): 
    for value in ids:
        condition = (df['id'] == value) & (df['model_name'] == key)
        # print('yes')
        counter+=1
        
        # print(len(df[condition]), value, key)
        
        df.loc[condition, 'human_verdict'] = 'correct'
print(counter)

In [ ]:

counter = 0
for key, ids in positives_to_negatives.items(): 
    for value in ids:
        condition = (df['id'] == value) & (df['model_name'] == key)
        # print('yes')
        
        # print(len(df[condition]))
        
        counter+=1
        df.loc[condition, 'human_verdict'] = 'incorrect'
        
print(counter)

In [ ]:
df['human_verdict'].value_counts()

In [28]:
df.to_csv('instruct_qa_natural_questions_corrected.csv', index=False)